# Deep Convolutional Generative Adversarial Network

Notebook adapted from [this site](https://www.tensorflow.org/tutorials/generative/dcgan)

This tutorial demonstrates how to generate images of handwritten digits using a [Deep Convolutional Generative Adversarial Network](https://arxiv.org/pdf/1511.06434.pdf) (DCGAN). The code is written using the [Keras Sequential API](https://www.tensorflow.org/guide/keras) with a `tf.GradientTape` training loop.

# What are GANs?

[Generative Adversarial Networks](https://arxiv.org/abs/1406.2661) (GANs) are one of the most interesting ideas in computer science today. Two models are trained simultaneously by an adversarial process. A *generator* ("the artist") learns to create images that look real, while a *discriminator* ("the art critic") learns to tell real images apart from fakes.

During training, the *generator* progressively becomes better at creating images that look real, while the *discriminator* becomes better at telling them apart. The process reaches equilibrium when the *discriminator* can no longer distinguish real images from fakes.

This notebook demonstrates this process on the MNIST dataset.

First download the [tensorflow library](https://www.tensorflow.org/) (if needed)  
`pip3 install --user --upgrade tensorflow`

Also run these commands to get the libraries needed to generate GIFs of your results  
`!pip install imageio`  
`!pip install git+https://github.com/tensorflow/docs`

In [ ]:
import tensorflow as tf
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
import PIL
from tensorflow.keras import layers
import time
from IPython import display
import tensorflow_docs.vis.embed as embed

# The _NEW_ Digits Dataset

Hopefully by now you are familiar with the digits data set we have been using. These 8x8 images are part of scikit-learn's digits data set. However, you may have noticed that many of these images do not clearly resemble actual digits!

Here we will use the MNIST dataset to train our GAN generator and discriminator. These digits are a bit larger - 28x28 - and will hopefully look more "real" than the digits we have been using. 

In [ ]:
# load the images into an array called "trainImages"
# load the digit labels into an array called "trainLabels"
(trainImages, trainLabels), (_, _) = tf.keras.datasets.mnist.load_data()
print(f"There are {len(trainImages)} digit images and labels")
print(f"Each image is {len(trainImages[0])} x {len(trainImages[0][0])} pixels")

In [ ]:
# We can use matplotlib's imshow function to display the data
rando = np.random.randint(len(trainImages))
print(f"Showing the {rando}th digit ({trainLabels[rando]})")
fig = plt.figure()
plt.imshow(trainImages[rando], cmap='gray')
plt.axis('off')
plt.show()

# Sorting and Cleaning the Data

First, notice that this data set contains 60,000 digits! This is much more than we need, so we will use a 1000-image subset to train the GAN.

To do this, we will use the provided `sortByNumber(digit, numSamples)` function. This function will return a subset of `numSamples` images of a single `digit` (0-9). Later, you may consider training your GAN on _all_ of the digits, but for now we will simply train our GAN to create the digit '2'

In [ ]:
# Let's filter out training data
def sortByNumber(digit, numSamples):
    """Returns an array of numSamples images of a digit
    Use digit = -1 to include all digits 0-9 inclusive
    Use numSamples = -1 for maximum value of numSamples"""
    
    numberImages = []
    if digit == -1:
        sortedData = trainImages
    else:
        for i in range(60000):
            if trainLabels[i] == digit:
                numberImages += [trainImages[i]]
        sortedData = np.asarray(numberImages)
        
    np.random.shuffle(sortedData)
    
    if 0 < numSamples < len(sortedData):
        sortedData = sortedData[:numSamples]
        
    return sortedData

In [ ]:
DIGIT = 2
NUM_SAMPLES = 1000
sortedImages = sortByNumber(DIGIT, NUM_SAMPLES)
print(f"There are {len(sortedImages)} training images")

Right now, ``sortedImages`` is a 3D array: each image is a 2D array and we have a whole list of images. However, the CNN functions used in TensorFlow take in 4D arrays as inputs! Therefore we will have to reshape the data by putting each single grayscale value into its own nested list.

In general, the 4D arrays are of the form (batchSize, height, width, channelSize)

<img src="input.png" width=300 />

- batchSize: the number of training inputs that are used each time the generator and discriminator are updated. For this example we will separate our 1000 images into batches of 50
- height: the height of the image. The training imgages have height 28
- width: the width of the image output. The training imgages have width 28
- channelSize: the complexity of the pixel data. For example, RGB pixels have channelSize 3. Since we are in grayscale, channelSize is simply 1

The current pixel data consists of integers on [0,255]. However, it will be better if we transform the pixel data to floating point numbers on [-1,1].

For the curious, ``BUFFER_SIZE`` is related to how we randomly order the training data. Using a buffer size equal to the number of training images is generally desirable because every image has an equal chance of being picked next in the final training data set. Check out [the docs](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#shuffle) to learn more

In [ ]:
BUFFER_SIZE = NUM_SAMPLES  # use perfectly random sampling among all 1000 digit images
BATCH_SIZE = 50   # train model on 50 images per loop

# create the 4D array!
cleanImageData = sortedImages.reshape(sortedImages.shape[0], 28, 28, 1).astype('float32')   # 3D -> 4D
cleanImageData = (cleanImageData - 127.5) / 127.5  # Normalize the images to [-1, 1]

# Batch and shuffle the data
trainingData = tf.data.Dataset.from_tensor_slices(cleanImageData).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

# The Big Picture

Both the generator and discriminator are defined using the [Keras Sequential API](https://www.tensorflow.org/guide/keras#sequential_model). The generator takes a random noise vector and turns it into an image. The discriminator takes an image input and turns it into a single number. Both accomplish this transformation by iteratively transforming their input's shape very slightly; these small transformations are called _layers_. They are trained simultaneously until the discriminator can no longer distinguish between real digits (training data) and fake digits (from the generator).

<img src="gist.png" width=300 />


# The Generator

The generator uses `Dense` and `Conv2DTranspose` layers to produce an image from a seed (a list of random numbers). The generator takes this list as input, then _upsamples_ several times until it reaches the desired image size of 28x28x1. _Upsampling_ is the process of constructing a larger image from a smaller image. More details can be found [here](https://towardsdatascience.com/types-of-convolutions-in-deep-learning-717013397f4d).

<img src="generator.png" width=300 />

In [ ]:
INPUT_SIZE = 100

def make_generator_model():
    model = tf.keras.Sequential()
    
    # add a Dense layer first
    model.add(layers.Dense(7*7*64, use_bias=False, input_shape=(INPUT_SIZE,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((7, 7, 64)))
    assert model.output_shape == (None, 7, 7, 64)  # None is the batch size

    # add a Conv2DTranspose layer
    model.add(layers.Conv2DTranspose(
        filters=32, 
        kernel_size=(3, 3), 
        strides=(1, 1), 
        padding='same', 
        use_bias=False))
    
    assert model.output_shape == (None, 7, 7, 32)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    # add another Conv2DTranspose layer
    model.add(layers.Conv2DTranspose(
        filters=8, 
        kernel_size=(3, 3), 
        strides=(2, 2), 
        padding='same', 
        use_bias=False))

    assert model.output_shape == (None, 14, 14, 8)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # add another Conv2DTranspose layer
    model.add(layers.Conv2DTranspose(
        filters=1, 
        kernel_size=(3, 3), 
        strides=(2, 2), 
        padding='same', 
        use_bias=False,
        activation='tanh'))
    
    assert model.output_shape == (None, 28, 28, 1)
    return model

# Understanding the Generator 

Let's take a closer look at the generator. Run the cell below to see some of the inner workings of it and its layers.

In [ ]:
generator = make_generator_model()
generator.summary()

Note that the argument `None` of the layers' input vectors refers to the batch size and will be filled in with the appropriate value when we actually start using the generator

Notice that our random noise input lists (length `INPUT_SIZE` = 100) are first transformed into 3136 element lists. These are then transformed into 7x7x64 images (7x7 pixels with 64 element lists for each pixel's data?!). Eventually these are transformed into the desired 28x28x1 images

# Testing the Generator

Let's test the generator's initial capabilities 

First we create a "noise" vector. This is a list floating point numbers (length = `INPUT_SIZE`). These values are randomly selected from a normal distribution (mean 0 and standard deviation 1). This acts as the input to our generator model. The generator takes in this 1D array input and does some fancy math to turn it into our desired 4D array output. 

In [ ]:
noise = tf.random.normal([1, INPUT_SIZE])   # a list of random floating point numbers
generated_image = generator(noise, training=False)

plt.imshow(generated_image[0, :, :, 0], cmap='gray')
plt.axis('off')
plt.show()

You might notice that the picture generated below does not resemble a digit. This is because the generator has not been trained. However, we have successfully transformed a 100 element 1D array into a 28x28 image!

# The Discriminator

The discriminator is a convolutional neural network (CNN) based image classifier. The discriminator works in the opposite direction of the generator. It uses `Conv2D` layers to take an image and transform it down into a single number output. The discriminator will be trained to output positive numbers for real images and negative numbers for fake images.

<img src="layers.png" width=300 />

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    
    model.add(layers.Conv2D(
        filters=32, 
        kernel_size=(3, 3), 
        strides=(2, 2), 
        padding='same',
        input_shape=[28, 28, 1]))
    
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(
        filters=64, 
        kernel_size=(3, 3), 
        strides=(2, 2), 
        padding='same'))
    
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

# Understanding the Discriminator

Let's take a closer look at the discriminator. Run the cell below.

In [ ]:
discriminator = make_discriminator_model()
discriminator.summary()

Again, the argument `None` of the layers' input vectors refers to the batch size and will be filled in with the appropriate value when we actually start using the discriminator

Notice that our images are first reshaped into 14x14x32 images (14x14 pixels with 32 element lists for each pixel's data). Eventually these are transformed into a single value

# Testing the Discriminator

Let's test the discriminator's initial capabilities 

We will pass in the `generated_image` from the untrained generator

In [ ]:
decision = discriminator(generated_image, training=False)
print(decision)

You might notice that the discriminator returns a value very close to 0. This is because the discriminator is also untrained and has no way to distinguish between real (positive output) and fake (negative output). However, we have successfully transformed a 28x28 image into a single number!

# Define the loss and optimizers

We can now define the functions that mathematically calculate how well the GAN is being trained. The below `cross_entropy` is a function that will calculate a measure of how different two distributions are. It will use the [logit](https://en.wikipedia.org/wiki/Logit) function to convert the positive/negative output from the discriminator to a probability from 0 to 1 (where 0 indicates a fake image and 1 represents a real image). Read more [here](https://towardsdatascience.com/understanding-binary-cross-entropy-log-loss-a-visual-explanation-a3ac6025181a)


In [ ]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

### Discriminator loss

This function quantifies how well the discriminator is able to distinguish real images from fakes. It compares the discriminator's predictions on real images to an array of 1s (because 1 corresponds to real images), and the discriminator's predictions on fake (generated) images to an array of 0s (because 0 corresponds to fake images). Ideally this `discriminator_loss` value will decrease as training occurs.

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

### Generator loss
The generator's loss quantifies how well it was able to trick the discriminator. Intuitively, if the generator is performing well, the discriminator will classify the fake images as real (or 1). Here, compare the discriminator's decisions on the generated images to an array of 1s.

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

### Adam

The Adam algorithm is simply a method of adjusting the model after training. Recall that neural nets are multivariable functions, and we can optimize such functions using the gradient. The Adam algorithm is a method of actually traversing the gradient. It will nudge the generator and discriminator in the right direction as training occurs.

The discriminator and the generator optimizers are different since you will train two networks separately.

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

# Define the training loop

The training loop begins with generator receiving a random seed as input. That seed is used to produce an image. The discriminator is then used to classify real images (drawn from the training set) and fakes images (produced by the generator). The loss is calculated for each of these models, and the gradients are used to update the generator and discriminator.

In fact, we can crudely study the GAN's progress by collecting data on the discriminator's performance. We will select a random training image and keep track of the discriminator's judgement on it as it gets trained. We will also select a random input to get passed through the generator and keep track of the discriminator's judgement on this fake image. Hopefully the discriminator's judgement on these two images will converge as the GAN is trained.

The `train_step` is the most fundamental function of the training loop and actually executed the process described above. Notice the use of `@tf.function`. This causes the code to switch from _eager_ execution to _graph_ execution. In short, this makes things run faster! If interested, more details can be found [here](https://towardsdatascience.com/eager-execution-vs-graph-execution-which-is-better-38162ea4dbf6)

In [ ]:
# choose a random generator input for data collection
testInput = tf.random.normal([1, INPUT_SIZE])

# choose a random real image for data collection
testImage = cleanImageData[np.random.randint(len(cleanImageData))].reshape(1, 28, 28, 1)

fakeError = []   # discriminator's judgement on the generator's fake image
realError = []   # discriminator's judgement on the real training image

In [ ]:
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, INPUT_SIZE])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

### Generate and save images

To visulize the GAN's progress, we will observe how 16 different random inputs are transformed by the generator as training occurs. Use the line below to create a folder to store these images.

The `generate_and_save_images` helper function will display and save the images so you can actually see how the training progresses over time.

In [ ]:
# Use the commented code below to _delete_ the folder
# shutil.rmtree('./generated_images')

# Use this line to make a subfolder to hold images
os.makedirs('./generated_images')

In [ ]:
def generate_and_save_images(model, epoch, test_input):
    # Notice `training` is set to False.
    # This is so all layers run in inference mode (batchnorm).
    predictions = model(test_input, training=False)

    fig = plt.figure(figsize=(4, 4))
    fig.suptitle(f"Epoch {epoch}")
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')

    plt.savefig('./generated_images/image_at_epoch_{:03d}.png'.format(epoch))
    plt.show()

## Train the model

The `train` function uses all of the aforementioned helpers to train the GAN over many iterations.

At the beginning of the training, the generated images look like random noise. As training progresses, the generated digits will look increasingly real!

The `epochs` parameter indicates how many iterations we will use. We will start by looping through all 1000 training images 50 times. 

This took about 145 seconds to run on my machine (who?). Hopefully it runs in similar time on yours

In [ ]:
def train(dataset, epochs):
    seed = tf.random.normal([16, INPUT_SIZE])
    fullStart = time.time()
    
    for epoch in range(epochs):
        start = time.time()
        for image_batch in dataset:
            train_step(image_batch)

        # Produce images for the GIF as you go
        display.clear_output(wait=True)
        generate_and_save_images(generator,epoch + 1,seed)

        print(f'Time for epoch {epoch + 1} is {time.time()-start} sec')
        
        global fakeError, realError
        fakeError += [discriminator(generator(testInput), training=False).numpy()[0][0]]
        realError += [discriminator(testImage, training=False).numpy()[0][0]]

    # Generate after the final epoch
    display.clear_output(wait=True)
    generate_and_save_images(generator,epochs,seed)
    print(f"Total run time is {time.time()-fullStart}")

In [ ]:
# THIS MAY TAKE A WHILE
train(trainingData, 100)

# Post-Training Analysis

The GAN has been trained! We can start analyzing its performance by having the generator create an image and having the discriminator judge it as real or fake. We can also have the discriminator judge a real image to see if it can actually tell the difference between real and fake. Remember that we want the discriminator to NOT tell the difference. Ideally the discriminator will output positive numbers for both real _and_ fake images

We can also view the data on the discriminator's judgement on the real and fake images.

In [ ]:
# Test real and fake images
noise = tf.random.normal([1, INPUT_SIZE])            # random generator input
indx = np.random.randint(NUM_SAMPLES)                # choose random real image to judge

generated_image = generator(noise, training=False)   # generator output

# DISCRIMINATOR JUDGEMENT
real_image = cleanImageData[indx].reshape(1, 28, 28, 1)
realImageDecision = discriminator(real_image)
fakeImageDecision = discriminator(generated_image)
print("Reminder that the discriminator gives positive values for real images and negative values for fake images")
print(f"Fake image: {fakeImageDecision[0][0]}")
print(f"Real image: {realImageDecision[0][0]}")

fig = plt.figure(figsize=(6, 3))
ax1 = fig.add_subplot(121)
ax1.imshow(generated_image[0, :, :, 0] * 127.5 + 127.5, cmap='gray')
ax1.axis('off')
ax1.title.set_text('Fake')
ax2 = fig.add_subplot(122)
ax2.imshow(real_image[0, :, :, 0] * 127.5 + 127.5, cmap='gray')
ax2.axis('off')
ax2.title.set_text('Real')

plt.show()

In [ ]:
# see the data we collected
epochTime = np.arange(1,100+1)
fig, ax = plt.subplots(figsize=(6, 6))
ax.plot(epochTime, realError)
ax.plot(epochTime, fakeError)
ax.legend(["Real Image", "Fake Image"])
ax.set_title('Discriminator Judgement vs Time')
ax.set_xlabel("Epoch")
ax.set_ylabel("Judgement")
plt.show()

# Images and GIFs

For convenience, the `display_image` function below will access the image at the specified epoch from the folder you created.

We can also create a GIF from these images for us to broadly judge how well the generator did

In [ ]:
# Display a single image using the epoch number
def display_image(epoch_no):
    return PIL.Image.open('./generated_images/image_at_epoch_{:03d}.png'.format(epoch_no))

display_image(1)   # type a number here to see the image generated at that epoch

In [ ]:
def createGIF():
    anim_file = 'dcgan.gif'
    with imageio.get_writer(anim_file, mode='I') as writer:
        filenames = glob.glob('./generated_images/image*.png')
        filenames = sorted(filenames)
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)
        image = imageio.imread(filename)
        writer.append_data(image)
    return anim_file

In [ ]:
embed.embed_file(createGIF())

# Variation of Parameters

Hopefully you found that to be very interesting! However, the example GAN above definitely could be improved.

Now it's your turn to train a GAN. Play around with the parameters in the cells below and re-train the model. Note that we have pasted the basic training function below because it needs to be freshly recompiled whenever you start training a new GAN. Be sure to run it too!

Note that training GANs can be tricky. It's important that the generator and discriminator do not overpower each other (e.g., that they train at a similar rate).

Adjust the parameters in the cell below, then run the other cells below to redo the training process

In [ ]:
# EDIT THIS CELL!!!
EPOCHS = 50                 # ~50 epochs recommended
noise_dim = 100             # between 20 and 300 recommended
DIGIT = -1                  # -1 for all digits, otherwise 0-9 inclusive
NUM_SAMPLES = 500           # -1 recommended if DIGIT == -1, otherwise <= 5000 recommended
BUFFER_SIZE = NUM_SAMPLES   # NUM_SAMPLES recommended
BATCH_SIZE = 20             # <= 10% of NUM_SAMPLES recommended

In [ ]:
# RE-COMPILE
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, INPUT_SIZE])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:
sortedImages = sortByNumber(DIGIT, NUM_SAMPLES)

# create the 4D array!
cleanImageData = sortedImages.reshape(sortedImages.shape[0], 28, 28, 1).astype('float32')   # 3D -> 4D
cleanImageData = (cleanImageData - 127.5) / 127.5  # Normalize the images to [-1, 1]

# data collection
testImage = cleanImageData[np.random.randint(len(cleanImageData))].reshape(1, 28, 28, 1)
testInput = tf.random.normal([1, INPUT_SIZE])
fakeError = []
realError = []

# Batch and shuffle the data
trainingData = tf.data.Dataset.from_tensor_slices(cleanImageData).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

generator = make_generator_model()
discriminator = make_discriminator_model()

# WARNING - we are deleting the folder of images from the previous run
shutil.rmtree('./generated_images')
os.makedirs('./generated_images')

if DIGIT != -1:
    print(f"There are {len(sortedImages)} images of the digit {DIGIT}")
else:
    print(f"There are {len(sortedImages)} images")
print(f"There are {len(trainingData)} batches of {BATCH_SIZE} images")

In [ ]:
train(trainingData, EPOCHS)

In [ ]:
# GIF
embed.embed_file(createGIF())

In [ ]:
# Test real and fake images
noise = tf.random.normal([1, INPUT_SIZE])            # random generator input
indx = np.random.randint(NUM_SAMPLES)                # choose random real image to judge

generated_image = generator(noise, training=False)   # generator output

# DISCRIMINATOR JUDGEMENT
real_image = cleanImageData[indx].reshape(1, 28, 28, 1)
realImageDecision = discriminator(real_image, training=False)
fakeImageDecision = discriminator(generated_image, training=False)
print("Reminder that the discriminator gives positive values for real images and negative values for fake images")
print(f"Fake image: {fakeImageDecision[0][0]}")
print(f"Real image: {realImageDecision[0][0]}")

fig = plt.figure(figsize=(6, 3))
ax1 = fig.add_subplot(121)
ax1.imshow(generated_image[0, :, :, 0] * 127.5 + 127.5, cmap='gray')
ax1.axis('off')
ax1.title.set_text('Fake')
ax2 = fig.add_subplot(122)
ax2.imshow(real_image[0, :, :, 0] * 127.5 + 127.5, cmap='gray')
ax2.axis('off')
ax2.title.set_text('Real')

plt.show()

In [ ]:
# see the data we collected
epochTime = np.arange(1,EPOCHS+1)
fig, ax = plt.subplots(figsize=(6, 6))
ax.plot(epochTime, realError)
ax.plot(epochTime, fakeError)
ax.legend(["Real Image", "Fake Image"])
ax.set_title('Discriminator Judgement vs Time')
ax.set_xlabel("Epoch")
ax.set_ylabel("Judgement")
plt.show()

This tutorial has shown the complete code necessary to write and train a GAN! To learn more about GANs see the [NIPS 2016 Tutorial: Generative Adversarial Networks](https://arxiv.org/abs/1701.00160).

If interested, feel free to explore some of the ideas below:

 - Experiment with a different dataset, for example the Large-scale Celeb Faces Attributes (CelebA) dataset [available on Kaggle](https://www.kaggle.com/jessicali9530/celeba-dataset). 

 - Feel free to adjust parameters in the actual `make_generator_model` and `make_discriminator_model` functions. 

   - [dropout](https://machinelearningmastery.com/dropout-for-regularizing-deep-neural-networks/) in the discriminator CNN

   - filter argument (see [docs](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2DTranspose))
   - kernel argument (see [docs](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2DTranspose))
   
Below are some links that I (who?) found useful 


[Tips On Training Your GANs Faster and Achieve Better Results](https://medium.com/intel-student-ambassadors/tips-on-training-your-gans-faster-and-achieve-better-results-9200354acaa5)

[Understanding Input Output shapes in Convolution Neural Network | Keras](https://towardsdatascience.com/understanding-input-and-output-shapes-in-convolution-network-keras-f143923d56ca)

[Generative Adversarial Network (GAN) for Dummies — A Step By Step Tutorial](https://towardsdatascience.com/generative-adversarial-network-gan-for-dummies-a-step-by-step-tutorial-fdefff170391)

[A Comprehensive Guide to Convolutional Neural Networks — the ELI5 way](https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53)